# 자연어 처리를 통해 딥러닝 환경 구현

Step 1 필요한 모듈로 부터 필요한 함수 임포트하기

In [1]:
import pandas as pd
import numpy as np
import json
import os
from konlpy.tag import Okt #강의 시간에 배운 konlpy를 이용해 불용어를 처리한다.


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

STEP2 데이터를 정리하기 

In [2]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

딥러닝에 방해되는 데이터(공백, Null)등을 제거하는 작업 

In [3]:
train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data['document'].replace('', np.nan, inplace=True) #빈 값을 NULL로 변경 
train_data = train_data.dropna(how = 'any') #Null 값 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") #한글과 공백을 제외한 모든 기호(특수 문자, 영어)를 제거하는 작업

In [4]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 

STEP 3 토큰화

In [5]:
okt = Okt() 
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

X_train = []
for sentence in train_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [6]:
X_test = []
for sentence in test_data['document']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

STEP 4 정수 인코딩

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

귀납적으로 분석하는 것이기에, 적은 빈도수가 나오는 단어들은 제거하는 작업을 거쳐 정확도를 높일 예정이다.

In [8]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
        
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
vocab_size = total_cnt - rare_cnt + 2


In [9]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [10]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

빈도 수가 적은 단어들을 제거했으므로 그 부분은 공백이 됬음을 의미함. 이 공백을 다시 제거해 딥러닝을 진행할 예정

In [11]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]
#길이가 0인 것들의 인덱스를 받아온다.

max_len=30
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

In [12]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
        
below_threshold_len(max_len, X_train)

In [13]:
X_train = pad_sequences(X_train, maxlen = 30)
X_test = pad_sequences(X_test, maxlen = 30)

# STEP 5 RNN을 응용한 LSTM 딥러닝 모델 구현하기

In [14]:
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128)) #:LSTM 층 형성 
model.add(Dense(1, activation='sigmoid'))

In [15]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model_a.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
#실행 후 최적의 모델을 a모델로 저장한다.

In [16]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
#optimizer 함수를 rms를 이용
#loss 함수는 교차 엔트로프 계열로 사용
# 모델 저장 이름 
model_name = input("저장할 모델의 이름을 저장하시오(예:model1.h5): ")
model.save(model_name)

저장할 모델의 이름을 저장하시오(예:model1.h5): model1.h5
